In [ ]:
import warnings
warnings.filterwarnings("ignore")

from carla.data.catalog import CsvCatalog
from carla import MLModelCatalog
from carla.recourse_methods import Clue, Wachter
from carla.models.negative_instances import predict_negative_instances
import numpy as np
# import torch

import sys
sys.path.insert(0,'..')
from recourse_util import update_dataset, train_recourse_method, predict, print_scores 

num = 10
# print(torch.cuda.is_available())

In [ ]:
def train_new_model(dataset):
    model = MLModelCatalog(dataset, "ann", load_online=False, backend="pytorch")
    model.train(
        learning_rate = 0.001,
        epochs = 10,
        max_depth = 50,
        n_estimators = 50,
        batch_size = 20,
        force_train = True
    )
    return model

In [ ]:
dataset = CsvCatalog(
    # file_path='datasets/bimodal_dataset_1.csv',
    file_path='datasets/unimodal_dataset_1.csv',
    # file_path='datasets/unimodal_dataset_2.csv',
    categorical=[],
    continuous=['feature1', 'feature2'],
    immutables=[],
    target='target'
)

In [ ]:
training_params = {"lr": 0.01, "epochs": 4, "batch_size": 1, "hidden_size": [20, 20]}

model = MLModelCatalog(
    dataset,
    model_type="ann",
    load_online=False,
    backend="pytorch"
)

model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)

In [ ]:
hyperparams = {
        "loss_type": "BCE"
    }

# load a recourse model and pass black box model
cl = Wachter(model, hyperparams)

In [ ]:
# generate counterfactual samples
factuals = predict_negative_instances(model, dataset._df).sample(num)
print("Number of factuals", len(factuals))
factuals

In [ ]:
counterfactuals = cl.get_counterfactuals(factuals)
print("Number of counterfactuals:", len(counterfactuals.dropna()))
counterfactuals

In [ ]:
counterfactuals

In [ ]:
post = dataset._df.copy()
update_dataset(post, factuals, counterfactuals)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(dataset._df['feature1'], dataset._df['feature2'], c=dataset._df['target'])
plt.show()

plt.scatter(post['feature1'], post['feature2'], c=post['target'])
plt.show()

# plt.scatter(factuals['feature1'], factuals['feature2'], c=factuals['target'])
# plt.show()

In [ ]:
from carla.evaluation.benchmark import Benchmark
benchmark = Benchmark(model, cl, factuals)
benchmark.run_benchmark()

In [ ]:
for i in range(10):
    print(cl.get_counterfactuals(factuals.iloc[:2]))

In [ ]:
hyperparams = {
                "data_name": 'custom',
                "train_vae": True,
                "width": 10,
                "depth": 3,
                "latent_dim": 12,
                "batch_size": 4,
                "epochs": 5,
                "lr": 0.0001,
                "early_stop": 20,
            }

cl = train_recourse_method(dataset, model, 'custom', 'CLUE', hyperparams)

for i in range(10):
    print(cl.get_counterfactuals(factuals.iloc[6:8]))